In [16]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
# from xgboost import XGBRegressor
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV


data_train = pd.read_parquet('/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Data/updated_standardization/data_set_train_tech_nonscaled.parquet')

data_test = pd.read_parquet('/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Data/updated_standardization/data_set_test_tech_nonscaled.parquet')

firm_data = pd.read_parquet('/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Data/Firm_variables/daily_firm_data_median_new.parquet')

# List of columns to drop
columns_to_drop = ['trading_days_till_exp'] + list(firm_data.columns[2:]) + ['moneyness_squared', 'tau_squared', 'moneyness_tau', 'best_offer_option', 'best_bid_option']
# # columns_to_drop = ['trading_days_till_exp']


# Drop columns from datasets if they exist
data_train = data_train.drop(columns=columns_to_drop, errors='ignore')
test_data = data_test.drop(columns=columns_to_drop, errors='ignore')

# Ensure that data_val and data_test have the same column order as data_train
data_test = data_test[data_train.columns]

data_trains = data_train.copy()

# data_train = data_trains[data_trains['date'] < '2020-01-01']
# data_validate = data_trains[data_trains['date'] >= '2020-01-01']

# (data_test)


In [17]:
# # Assuming 'data_train' and 'data_test' are already defined and loaded

# # Top features for Call and Put options
# # Top features for Call and Put options
# top_features_c = ['cp_flag', 'Ticker', 'date', 'impl_volatility', 'prev_day_iv', 'T', 'volume_option', 'prev2_day_iv', 'gold_price']
# top_features_p = ['cp_flag', 'Ticker', 'date', 'impl_volatility', 'prev_day_iv', 'T', 'volume_option', 'prev2_day_iv', '5_day_rolling_return_stock']

top_features_c =['cp_flag', 'Ticker', 'date', 'impl_volatility','prev_day_iv', 'T', 'vol_stock', 'prev2_day_iv', 'hi-lo_stock', '1Y_bond', 'spread_stock', 'spread_vix', 'moneyness', '5_day_rolling_return_stock']
top_features_p =['cp_flag', 'Ticker', 'date', 'impl_volatility','prev_day_iv', 'T', 'vol_stock', 'prev2_day_iv', 'hi-lo_stock', '5_day_rolling_return_stock', 'RET', 'moneyness', 'PRC_actual', 'CLOSE_vix']


# top_features_c = ['cp_flag', 'Ticker', 'date', 'impl_volatility', 'prev_day_iv', 'T', 'vol_stock', 'prev2_day_iv', 'hi-lo_stock', '1Y_bond']
# top_features_p = ['cp_flag', 'Ticker', 'date', 'impl_volatility', 'prev_day_iv', 'T', 'vol_stock', 'prev2_day_iv', 'hi-lo_stock']


# # Prepare train data for Call and Put options
data_train_c = data_train[data_train['cp_flag'] == 'C'][top_features_c]
data_train_p = data_train[data_train['cp_flag'] == 'P'][top_features_p]

# # Prepare validation data for Call and Put options
# # data_validate_c = data_validate[data_validate['cp_flag'] == 'C'][top_features_c]
# # data_validate_p = data_validate[data_validate['cp_flag'] == 'P'][top_features_p]

# # Prepare test data for Call and Put options
data_test_c = data_test[data_test['cp_flag'] == 'C'][top_features_c]
data_test_p = data_test[data_test['cp_flag'] == 'P'][top_features_p]

# # data_test_c

In [14]:
# # # Prepare train data for Call and Put options
# data_train_c = data_train[data_train['cp_flag'] == 'C']
# data_train_p = data_train[data_train['cp_flag'] == 'P']

# # # Prepare validation data for Call and Put options
# # # data_validate_c = data_validate[data_validate['cp_flag'] == 'C'][top_features_c]
# # # data_validate_p = data_validate[data_validate['cp_flag'] == 'P'][top_features_p]

# # # Prepare test data for Call and Put options
# data_test_c = data_test[data_test['cp_flag'] == 'C']
# data_test_p = data_test[data_test['cp_flag'] == 'P']

In [21]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
import numpy as np
import random
import tensorflow as tf

# Set random seeds for reproducibility
def set_seed(seed=42):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)

# Set the seed before training
set_seed(42)

def prepare_data_with_gridsearch(train_data, test_data, option_type):
    """
    Prepare the data, train LightGBM using GridSearchCV, and evaluate both in-sample and out-of-sample performance.
    
    Parameters:
    train_data (pd.DataFrame): The training dataset.
    test_data (pd.DataFrame): The testing dataset.
    option_type (str): Call or Put option type for labeling the print output.
    """
    # Prepare the train and test data
    X_train = train_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Training features
    y_train = train_data['impl_volatility']  # Training target
    
    X_test = test_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Test features
    y_test = test_data['impl_volatility']  # Test target

    # Define parameter grid for GridSearchCV with regularization terms
    param_grid = {
        'n_estimators': [50, 100, 150, 200],
        'learning_rate': [0.001, 0.01, 0.1],
        'max_depth': [ 20],
        'num_leaves': [  50],
    }

    # param_grid = {'colsample_bytree': [1.0], 'learning_rate': [0.1], 'max_depth': [10], 'n_estimators': [100], 'num_leaves': [63], 'subsample': [0.6]}

    # Initialize the LGBMRegressor
    lgbmodel = LGBMRegressor()

    # Perform GridSearchCV to find the best hyperparameters
    grid_search = GridSearchCV(estimator=lgbmodel, param_grid=param_grid, 
                               cv=3, n_jobs=-1, verbose=2)

    print(f"Running GridSearchCV for {option_type} Options...")
    grid_search.fit(X_train, y_train)

    # Get the best parameters and use them for the final model
    best_params = grid_search.best_params_
    print(f"\nBest Parameters for {option_type} Options: {best_params}")

    # Initialize the best model with early stopping manually
    best_LGBM_model = LGBMRegressor(**best_params)

    print(f"\nTraining the model with early stopping for {option_type} Options...")
    best_LGBM_model.fit(X_train, y_train, 
                        eval_set=[(X_test, y_test)])

    # In-sample (training set) predictions
    y_train_pred = best_LGBM_model.predict(X_train)

    # Out-of-sample (test set) predictions
    y_test_pred = best_LGBM_model.predict(X_test)

    # Evaluate In-Sample Performance (on Training Data)
    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
    r2_train = r2_score(y_train, y_train_pred)

    # Evaluate Out-of-Sample Performance (on Test Data)
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    r2_test = r2_score(y_test, y_test_pred)

    # Print the results
    print(f"\nPerformance for {option_type} Options:")
    print(f"In-Sample RMSE (Training): {rmse_train:.4f}")
    print(f"In-Sample R² (Training): {r2_train:.4f}")
    print(f"Out-of-Sample RMSE (Test): {rmse_test:.4f}")
    print(f"Out-of-Sample R² (Test): {r2_test:.4f}")

    # Ensure in-sample R² is less than 0.9
    if r2_train >= 0.9:
        print(f"Warning: In-Sample R² is too high ({r2_train:.4f}). Consider increasing regularization or reducing model complexity.")

    return best_LGBM_model

# Call the function for Call options data
best_model_call = prepare_data_with_gridsearch(data_train_c, data_test_c, 'Call')

# Call the function for Put options data (uncomment to run)
best_model_put = prepare_data_with_gridsearch(data_train_p, data_test_p, 'Put')


Running GridSearchCV for Call Options...
Fitting 3 folds for each of 12 candidates, totalling 36 fits
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004083 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2268
[LightGBM] [Info] Number of data points in the train set: 48231, number of used features: 10
[LightGBM] [Info] Start training from score 0.654958
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001139 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2275
[LightGBM] [Info] Number of data points in the train set: 48231, number of used features: 10
[LightGBM] [Info] Start training from score 0.596373
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002759 seconds.
You can set `force_row_wise=true` to remove th